### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
# ligas = temporada[temporada['Date'] == data_var]['League'].unique().tolist()
# temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos[['Date', 'League', 'Time', 'Home', 'Away', 
       'Odds_H', 'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', ]]

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A
5423,2023-10-30,Eua Nba,20:00,Charlotte Hornets,Brooklyn Nets,2.00,1.83,226.5,1.91,1.91,1.5,1.91,1.91
5424,2023-10-30,Eua Nba,20:30,Atlanta Hawks,Minnesota Timberwolves,2.15,1.74,234.5,1.91,1.91,2.5,1.91,1.91
5425,2023-10-30,Eua Nba,21:00,Memphis Grizzlies,Dallas Mavericks,2.15,1.74,227.5,1.91,1.91,2.5,1.91,1.91
5426,2023-10-30,Itália Liga A,16:30,Basket Napoli,Virtus Bologna,3.20,1.37,162.5,1.91,1.91,6.5,1.91,1.91
5427,2023-10-30,Alemanha Bbl,14:30,Gottingen,Alba Berlin,4.75,1.20,169.5,1.91,1.91,9.5,1.91,1.91
5428,2023-10-30,Argentina Liga A,21:00,Zarate,Ferro,2.30,1.56,155.5,1.83,1.83,3.5,1.83,1.83
5429,2023-10-30,China Cba,08:35,Beijing Royal Fighters,Jiangsu Dragons,1.20,4.60,202.5,1.87,1.87,-9.5,1.87,1.87
5430,2023-10-30,China Cba,08:35,Fujian,Shanghai,2.90,1.43,212.5,1.87,1.87,4.5,1.87,1.87
5431,2023-10-30,China Cba,08:35,Nanjing Tongxi,Shenzhen,3.20,1.36,202.5,1.87,1.87,6.5,1.87,1.87
5432,2023-10-30,Coreia Do Sul Kbl,07:00,Suwon KT,Wonju DB,2.35,1.65,162.5,1.91,1.91,2.5,1.91,1.91


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,20:00,Eua Nba,Charlotte Hornets,Brooklyn Nets,226.5,1.91,0.7846,Over
1,20:30,Eua Nba,Atlanta Hawks,Minnesota Timberwolves,234.5,1.91,0.7688,Over
2,21:00,Eua Nba,Memphis Grizzlies,Dallas Mavericks,227.5,1.91,0.7753,Over
3,16:30,Itália Liga A,Basket Napoli,Virtus Bologna,162.5,1.91,0.8387,Over
4,14:30,Alemanha Bbl,Gottingen,Alba Berlin,169.5,1.91,0.8443,Over
9,07:00,Coreia Do Sul Kbl,Suwon KT,Wonju DB,162.5,1.91,0.8274,Over
10,15:15,Grécia Liga De Basquete,Olympiakos,Panathinaikos,151.5,1.91,0.8471,Over
12,21:00,Eua Nba,New Orleans Pelicans,Golden State Warriors,224.5,1.91,0.7804,Over
13,22:00,Eua Nba,Denver Nuggets,Utah Jazz,227.5,1.91,0.7925,Over


### AH Zebra V1 (LR)

In [9]:
# Duplicando o dataset
df_ah_pos_v1 = jogos.copy()

# AH
df_ah_pos_v1.loc[(df_ah_pos_v1['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_ah_pos_v1.loc[(df_ah_pos_v1['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_ah_pos_v1['Zebra'] = df_ah_pos_v1['Zebra'].astype('category')
# Odds
df_ah_pos_v1.loc[(df_ah_pos_v1['Zebra']) == 'Casa', 'Odds'] = df_ah_pos_v1['HA_Odds_H']
df_ah_pos_v1.loc[(df_ah_pos_v1['Zebra']) == 'Fora', 'Odds'] = df_ah_pos_v1['HA_Odds_A']

df_ah_pos_v1.reset_index(drop=True, inplace=True)
df_ah_pos_v1.replace([np.inf, -np.inf], 0, inplace=True)

if not df_ah_pos_v1.empty:
    mdl_ah_pos_v1 = load_model('ML/ah_pos/v1/v1_ah_pos', verbose=False)
    prev = predict_model(mdl_ah_pos_v1, df_ah_pos_v1)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds'] > 1.85))]
    prev = prev[((prev['Over_Line'] >= 200) & (prev['Over_Line'] < 212)) | ((prev['Over_Line'] >= 215) & (prev['Over_Line'] < 218))]

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/ah_pos_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO AH POSITIVO V1 (LR) ------------')
        print()

        prev['Bet'] = 'AH Zebra'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO AH POSITIVO V1 (LR) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
6,08:35,China Cba,Beijing Royal Fighters,Jiangsu Dragons,-9.5,1.87,0.5147,AH Zebra


### AH Zebra V2 (ADA)

In [10]:
# Duplicando o dataset
df_ah_pos_v2 = jogos.copy()

# AH
df_ah_pos_v2.loc[(df_ah_pos_v2['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_ah_pos_v2.loc[(df_ah_pos_v2['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_ah_pos_v2['Zebra'] = df_ah_pos_v2['Zebra'].astype('category')
# Odds
df_ah_pos_v2.loc[(df_ah_pos_v2['Zebra']) == 'Casa', 'Odds'] = df_ah_pos_v2['HA_Odds_H']
df_ah_pos_v2.loc[(df_ah_pos_v2['Zebra']) == 'Fora', 'Odds'] = df_ah_pos_v2['HA_Odds_A']

df_ah_pos_v2.reset_index(drop=True, inplace=True)
df_ah_pos_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_ah_pos_v2.empty:
    mdl_ah_pos_v2 = load_model('ML/ah_pos/v2/v2_ah_pos', verbose=False)
    prev = predict_model(mdl_ah_pos_v2, df_ah_pos_v2)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds'] >= 1.85))]
    prev = prev[((prev['HA_Line'] <= -7)) & ((prev['HA_Line'] >= -13))]
    
    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/ah_pos_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO AH POSITIVO V2 (ADA) ------------')
        print()

        prev['Bet'] = 'AH Zebra'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO AH POSITIVO V2 (ADA) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
6,08:35,China Cba,Beijing Royal Fighters,Jiangsu Dragons,-9.5,1.87,0.5021,AH Zebra
10,15:15,Grécia Liga De Basquete,Olympiakos,Panathinaikos,-7.5,1.91,0.6337,AH Zebra
13,22:00,Eua Nba,Denver Nuggets,Utah Jazz,-8.5,1.91,0.5003,AH Zebra


### AH Zebra V3 (ADA - SEED 8941)

In [11]:
# Duplicando o dataset
df_ah_pos_v3 = jogos.copy()

# AH
df_ah_pos_v3.loc[(df_ah_pos_v3['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_ah_pos_v3.loc[(df_ah_pos_v3['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_ah_pos_v3['Zebra'] = df_ah_pos_v3['Zebra'].astype('category')
# Odds
df_ah_pos_v3.loc[(df_ah_pos_v3['Zebra']) == 'Casa', 'Odds'] = df_ah_pos_v3['HA_Odds_H']
df_ah_pos_v3.loc[(df_ah_pos_v3['Zebra']) == 'Fora', 'Odds'] = df_ah_pos_v3['HA_Odds_A']

df_ah_pos_v3.reset_index(drop=True, inplace=True)
df_ah_pos_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_ah_pos_v3.empty:
    mdl_ah_pos_v3 = load_model('ML/ah_pos/v3/v3_ah_pos', verbose=False)
    prev = predict_model(mdl_ah_pos_v3, df_ah_pos_v3)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds'] >= 1.85))]
    prev = prev[((prev['HA_Line'] <= -7)) & ((prev['HA_Line'] >= -13))]
    
    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/ah_pos_v3_{data_var}.csv', index=False)

        print('------------ MÉTODO AH POSITIVO V3 (ADA - SEED 8941) ------------')
        print()

        prev['Bet'] = 'AH Zebra'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO AH POSITIVO V3 (ADA - SEED 8941) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
13,22:00,Eua Nba,Denver Nuggets,Utah Jazz,-8.5,1.91,0.5,AH Zebra


### AH Zebra V4 (ADA - SEED 4280)

In [12]:
# Duplicando o dataset
df_ah_pos_v4 = jogos.copy()

# AH
df_ah_pos_v4.loc[(df_ah_pos_v4['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_ah_pos_v4.loc[(df_ah_pos_v4['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_ah_pos_v4['Zebra'] = df_ah_pos_v4['Zebra'].astype('category')
# Odds
df_ah_pos_v4.loc[(df_ah_pos_v4['Zebra']) == 'Casa', 'Odds'] = df_ah_pos_v4['HA_Odds_H']
df_ah_pos_v4.loc[(df_ah_pos_v4['Zebra']) == 'Fora', 'Odds'] = df_ah_pos_v4['HA_Odds_A']

df_ah_pos_v4.reset_index(drop=True, inplace=True)
df_ah_pos_v4.replace([np.inf, -np.inf], 0, inplace=True)

if not df_ah_pos_v4.empty:
    mdl_ah_pos_v4 = load_model('ML/ah_pos/v3/v4_ah_pos', verbose=False)
    prev = predict_model(mdl_ah_pos_v4, df_ah_pos_v4)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds'] >= 1.85))]
    prev = prev[((prev['HA_Line'] <= -7)) & ((prev['HA_Line'] >= -13))]
    
    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/ah_pos_v4_{data_var}.csv', index=False)

        print('------------ MÉTODO AH POSITIVO V4 (ADA - SEED 4280) ------------')
        print()

        prev['Bet'] = 'AH Zebra'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO AH POSITIVO V4 (ADA - SEED 4280) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
6,08:35,China Cba,Beijing Royal Fighters,Jiangsu Dragons,-9.5,1.87,0.5015,AH Zebra
10,15:15,Grécia Liga De Basquete,Olympiakos,Panathinaikos,-7.5,1.91,0.5870,AH Zebra
13,22:00,Eua Nba,Denver Nuggets,Utah Jazz,-8.5,1.91,0.5041,AH Zebra
